<a href="https://colab.research.google.com/github/GregKyri/Manganese-from-source-to-tap/blob/main/aeration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Aeration***


In [ ]:
import math as mh
import pandas as pd
import numpy as np

def aeration(lanes,Partable2,dailyflow,Treatment):
    # Aeration stage in the groundwater systems  design
    # This code calculates the dimensions of aeration treatment stage in a water treatment plant when ground water is the source of water.
    #treatmenttechnologies2 = treatmenttechnologies.astype(str)

    # Input parameters
    print("")
    print("AERATION PROCESS")
    flowRate = max(dailyflow)/24  # m3/h
    DO = Partable2['Dissolved Oxygen (DO)'][2]
    #DO=2
    print("The total Dissolved Oxygen (DO) in the source water is",DO)
    # Select method
    if (Treatment['Treatment Technologies'].str.lower().str.strip() == "cascades").any():
      aerarea,aerenergy =cascades(flowRate,lanes,DO) #, units
    elif (Treatment['Treatment Technologies'].str.lower().str.strip() == "spray aeration").any():
      aerarea,aerenergy=spray(flowRate,lanes,DO) #, units
    elif (Treatment['Treatment Technologies'].str.lower().str.strip() == "stripping tower").any():
      aerarea,aerenergy=stripping(flowRate,lanes,DO) #, units
    elif (Treatment['Treatment Technologies'].str.lower().str.strip() == "plate aeration").any():
      aerarea,aerenergy=plate(flowRate,lanes,DO) #, units
    else:
      aerarea = 0
      aerenergy = 0

    return aerarea,aerenergy

def get_E(h):
    """Returns oxygen transfer efficiency per step (E) of a cascade based on step height (h)."""
    E_table = {
        0.2: 0.14,
        0.4: 0.25,
        0.6: 0.36,
        0.8: 0.46,
        1.0: 0.51,
        1.2: 0.55
    }
    return E_table.get(h, 0.30)  # Default to 0.3 if h is not in the table

def cascades(flowRate,lanes,DO):
    units=float(input("Enter the number of units per lane: "))
    flowlane=flowRate/lanes # m3/h
    flowunit=flowRate/(lanes*units) # m3/h

    h = float(input("Enter weir height (m - values from 0.2 to 1.2 with 0.2 step):"))
    N=int(input("Set the number of cascade steps(1-5):"))
    H = float(input("Se the weir height (m) - minimum H=2*h where h is the weir height:"))

    # Compute total oxygen transfer efficiency (OTE)
    E = get_E(h)  # Get oxygen transfer efficiency
    OTE = 1 - (1 - E) ** N

    # Compute DO increase
    DO_sat=10 #saturated DO is 10mg/l
    DO_increase = (DO_sat - DO) * OTE
    DO_final = DO + DO_increase
    print("The DO (mg/l) in the cascade outlet is",DO_final)

    # Calculate time and velocity
    t=mh.sqrt(2*h/9.81) #sec

    # Calculate Height of the tank
    Hall=1.1*(H*N+h)

    # Calculate Length
    tank_num=int(input("Set the number of parts per cascade tank:"))
    part_length=float(input("Length of each part (m):"))
    Lnet=tank_num*part_length
    print("The total net weir length (m) is:",Lnet)
    Lall=mh.ceil(Lnet+0.4*(1+tank_num))
    print("The legth of the weir is ():",Lall)

    # Calculate width
    d=((flowunit ** 2) / (9.81 * Lnet ** 2))**(1/3) #thickness of the falling jet
    v=flowunit/(Lnet*d) # velocity of the falling water
    x=v*t # distance of the falling water
    W=mh.ceil(2*x) #width of tank
    print("The width of each cascade tank (m) is:",W)
    Wall=mh.ceil(N*W+(N+1)*0.4)

    # Calculate the area
    aerareaunit=mh.ceil(Wall*Lall+(5*Lall))
    print("The area per unit(m2), assuming 5m channel flow is:",aerareaunit)
    aerarea=mh.ceil(aerareaunit*lanes*units)
    print("The total aerated area (m2) is:",aerarea)

    # Calculate energy consumption
    print("The head required for cascades (m) is:",Hall)
    Plane=1.1*(flowlane*Hall)/(367*0.75) # assuming efficiency of 0.75
    aerenergy=mh.ceil.(lanes*Plane*24)
    print("The total aeration energy consumption per day (kWh) is:",aerenergy)
    return aerarea,aerenergy

def spray(flowRate,lanes,DO):
    units=float(input("Enter the number of units per lane: "))
    flowlane=flowRate/lanes # m3/h
    flowunit=flowlane/units # m3/h
    flowunitlpermin=flowunit*16.67 #m3/h to l/min
    print("For spray aeration we assume that full cone 90 degrees spray nozzles are used")
    Hpres=float(input("Set available pressure in bars (1-5):"))

    # BETE TF Series Nozzle Flow Rates (m³/h) at Different Pressures (bar)
    bete_tf_nozzles = {
        "TF6": {1: 3.19, 2: 4.5,  3: 5.5,  4: 6.1,  5: 7.1},
        "TF8": {1: 5.93, 2: 8.4,  3: 10.3, 4: 12.0, 5: 13.2},
        "TF10": {1: 9.12, 2: 12.9, 3: 15.8, 4: 18.5, 5: 20.4},
        "TF12": {1: 13.7, 2: 19.3, 3: 23.7, 4: 27.0, 5: 30.6},
        "TF14": {1: 18.5, 2: 26.1, 3: 32.0, 4: 36.5, 5: 41.3},
        "TF16": {1: 24.2, 2: 34.2, 3: 41.8, 4: 47.2, 5: 54.0},
    }

    if Hpres not in [1, 2, 3, 4, 5]:
        print("Error: Nozzle data is available only for 1 to 5 bar.")
        return None, None

    # Get available nozzles for the given pressure
    available_nozzles = {nozzle: flow[Hpres] for nozzle, flow in bete_tf_nozzles.items()}

    # Find all possible nozzle combinations
    best_config = None
    min_nozzles = float('inf')
    best_diff = float('inf')

    for nozzle, flow_rate in available_nozzles.items():
        for num_nozzles in range(1, 11):  # Try up to 10 nozzles
            total_flow = num_nozzles * flow_rate
            flow_diff = abs(total_flow - flowunitlpermin)

            if flow_diff < best_diff or (flow_diff == best_diff and num_nozzles < min_nozzles):
                best_config = (num_nozzles, nozzle, total_flow)
                best_diff = flow_diff
                min_nozzles = num_nozzles

    if best_config:
        num_nozzles, nozzle, total_flow = best_config
        print(f"Optimal Solution per unit: {num_nozzles} × {nozzle} Nozzles (Total Flow: {total_flow:.2f} m³/h at {Hpres} bar)")
    else:
        print("No suitable nozzle configuration found.")
        return None, None

    # Calculate tank aerea
    unitsprayarea= 2*num_nozzles # 2m2 per nozzle
    laneprayarea=mh.ceil(unitsprayarea*lanes)
    aerarea=mh.ceil(laneprayarea*lanes)

    print("The total aeration process area (m2) per lane is:",laneprayarea)
    print("The total aeration process area (m2) is:",aerarea)

    # Calculate energy consumption
    Hhead=1.15*Hpres
    print("The head required for spray aeration (m) is:",Hhead)
    Plane=1.1*(flowlane*Hhead/(367*0.75)) # assuming efficiency of 0.7
    aerenergy=mh.ceil(lanes*Plane*24)
    print("The total aeration energy consumption per day (kWh) is:",aerenergy)
    return aerarea,aerenergy

def stripping(flowRate,lanes,DO):
    units=float(input("Enter the number of units per lane: "))
    flowlane=flowRate/lanes # m3/h
    flowunit=flowlane/units # m3/h
    """
    Selects an appropriate stripping tower based on the input flow rate.
    """
    towers = [
        {"diameter": 0.3, "height": 3, "flow_min": 2, "flow_max": 20},
        {"diameter": 0.9, "height": 6, "flow_min": 20, "flow_max": 83},
        {"diameter": 1.5, "height": 8, "flow_min": 83, "flow_max": 208},
        {"diameter": 2.5, "height": 10, "flow_min": 208, "flow_max": 625},
        {"diameter": 4.0, "height": 12, "flow_min": 625, "flow_max": 2083},
    ]

    # Find the matching
    selected_tower = None
    for tower in towers:
        if tower["flow_min"] <= flowunit <= tower["flow_max"]:
            selected_tower = tower
            break
    if selected_tower:
        towerdiam=selected_tower['diameter']
        towerheight=selected_tower['height']
        Atower=3.14*towerdiam**2/4 # m2
        print(f"Recommended Stripping Tower:\nDiameter: {tower['diameter']} m\nHeight: {tower['height']} m\nTower Area:{Atower}m2")
    else:
        print("No suitable tower found. Consider custom design.")
        return None, None # Exit function

    # Compute DO increase and air requirement
    DO_sat=float(input("Desired DO (mg/l) in the outlet of aeration:"))
    DO_increase = DO_sat - DO

    # Calculate the blowers' characteristics
    print("Assuming 1+1 blower per lane")
    OTR=flowlane*DO_increase/1000 # Oxygen transfer rater (OTR) in kg/h
    SOTR= OTR/0.95 #Standard Oxygen Transfer Rate (SOTR) in kg/h.Salinity correction factor for freshwater is assumed 0.95
    Qair=mh.ceil(SOTR/(0.21 *1.429)) # m3/h assuming 21% oxygen in the air and the O2 density being 1.429kg/m3
    print(f"Required air mass per stripping tower: {SOTR/2:.2f} kg/h")
    print(f"Required blower flow: {Qair:.2f} m³/h")
    Pamb=101.325 #N/m2
    Pdrop=50*towerheight # Assuming 50N/m2 pressure dropdrop per meter (MWH's Water treatment - Principles and Design)
    kp=275 # N*s^2/m^4 empiricaal constant (MWH's Water treatment - Principles and Design)
    Plosses=kp*(Qair/Atower)**2
    Pin=1.1*(Pamb+Pdrop+Plosses) # N/m2
    Pinbar=mh.ceil(Pin/100) # N/m2 to mbar
    Pambbar=Pamb/100 # N/m2 to mbar
    R=8.314 #J/mol*K universal gas constant
    Tair=273+15 #absolute temperature in K,assuming 15C air temerature
    MW=28.97 #g/mol molecular weight of air
    na=0.283 # blower break power for air (MWH's Water treatment - Principles and Design)
    nef=0.4 # assuming blower's efficiency
    Pblo=mh.ceil((SOTR*R*Tair/MW*na*nef)*((Pin/Pamb)**na-1)) # KW
    Eblo=mh.ceil(Pblo*lanes*24)
    blowarea=2*(lanes*2) # assuming 2m2 per lane based on avarage dimensions of blowers

    # Blower characteristics
    print("Total number of  blowers (including stand-by):",lanes*2)
    print("Blower flow (m3/h) :",Qair, "m3/h")
    print("Blower Pressure:", Pinbar, "mbar")
    print("Blower Power:", Pblo, "kW")
    print("Total blowers energy consumption per day:", Eblo, "kWh/day")

    #Calculate the pumps' characteristics
    print("Assuming 2 (1+1 stand-by) pumps per lane")
    Qpump=flowlane
    Hpump=1.1*(towerheight+2) #assuming 2m of losses in frictions
    Ppump=1.1*(Qpump*Hpump/(367*0.75)) #Kw
    Epump=mh.ceil(Ppump*lanes*24)
    pumparea=4*(lanes*2) # assuming 4m2 per lane based on avarage dimensions of pumps

    # Pumps characteristics
    print("Total number of  pumps (including stand-by):",lanes*2)
    print("Pump flow (m3/h) :",Qpump, "m3/h")
    print("Pump Pressure:", Hpump, "mH2O")
    print("Pump Power:", Ppump, "kW")
    print("Total pumps energy consumption per day:", Epump, "kWh/day")

    # Total aerea required
    aerarea=mh.ceil(blowarea+pumparea+1.1*(units*lanes*Atower))
    print("The total area of the aeration process (m2) is:",aerarea)

    # Total required energy
    aerenergy=mh.ceil(Eblo+Epump)
    print("The total energy consumption (kWh) per day is:",aerenergy)

    return aerarea,aerenergy

def plate(flowRate,lanes,DO):
    print("Assuming one aeration basin per lane")
    flowlane=flowRate/lanes # m3/h

    """
    Calculates the required basin area, number of plate aerators, and other parameters for a groundwater treatment plant.

    Args:
        flowRate: Flow rate of the treatment plant in m3/day.
        DO: Initial dissolved oxygen concentration in mg/L.
        DO_sat: Desired dissolved oxygen concentration in mg/L.
        hrt: Hydraulic retention time in hours.
        plate_aerator_capacity: Oxygen transfer capacity of a single plate aerator in kg/h = 6 kg/h.
        o2ef: the oxygen transfer efficeincy

    Returns:
        A dictionary containing the calculated values:
            - basin_area: Basin area in m2.
            - basin_volume: Basin volume in m3.
            - oxygen_demand: Oxygen demand in kg/day.
            - required_aerators: Number of plate aerators required.
    """

    # Compute DO increase and air requirement
    DO_out=float(input("Desired DO (mg/l) in the outlet of aeration:"))
    DO_s=10 #saturation dissolved oxygen  concentration
    DO_increase = DO_out - DO

    # Calculate the blowers' characteristics
    print("Assuming 1+1 blower per lane")
    OTR=flowlane*DO_increase/1000 # Oxygen transfer rater (OTR) in kg/h
    o2ef=0.6 # 0.6 is the O2 transfer efficiency (given by the plate aerator manufacturer)
    SOTR= OTR/(o2ef *0.95) #Standard Oxygen Transfer Rate (SOTR) in kg/h. Salinity correction factor for freshwater is assumed to be 0.95
    Qair=mh.ceil(SOTR/(0.21 *1.429)) # m3/h assuming 21% oxygen in the air and the O2 density being 1.429kg/m3
    print(f"Required air mass per lane: {SOTR:.2f} kg/h")
    print(f"Required blower flow: {Qair:.2f} m³/h")
    Pamb=101.325 #N/m2
    Pdrop=50*5 # Assuming 50N/m2 pressure dropdrop per meter (MWH's Water treatment - Principles and Design)
    Pin=1.1*(Pamb+Pdrop) # N/m2
    Pinbar=mh.ceil(Pin/100) # N/m2 to mbar
    Pambbar=Pamb/100 # N/m2 to mbar
    R=8.314 #J/mol*K universal gas constant
    Tair=273+15 #absolute temperature in K,assuming 15C air temerature
    MW=28.97 #g/mol molecular weight of air
    na=0.283 # blower break power for air (MWH's Water treatment - Principles and Design)
    nef=0.4 # assuming blower's efficiency
    Pblo=mh.ceil((SOTR*R*Tair/MW*na*nef)*((Pin/Pamb)**na-1)) # KW
    Eblo=mh.ceil(Pblo*lanes*24)
    blowarea=2*(lanes*2) # assuming 2m2 per lane based on avarage dimensions of blowers

    # Blower characteristics
    print("Total number of  blowers (including stand-by):",lanes*2)
    print("Blower flow (m3/h) :",Qair, "m3/h")
    print("Blower Pressure:", Pinbar, "mbar")
    print("Blower Power:", Pblo, "kW")
    print("Total blowers energy consumption per day:", Eblo, "kWh/day")

    # Calculate required oxygen transfer rate per aerator
    plate_aerator_capacity= 6 #kg/h according to the manufacturer
    print("The capacity of the plate aerator (kg/h) is",plate_aerator_capacity)
    required_oxygen_transfer_rate_per_aerator = SOTR / (24 * plate_aerator_capacity)
    required_aerators = mh.ceil(required_oxygen_transfer_rate_per_aerator)
    print("The number of plate aerators per lane is",required_aerators)
    print("The total number of plate aerators is",2*required_aerators)

    # Calculate the basin area
    hrt=float(input("Set the retention time(h):"))
    basin_volume = flowlane * hrt
    basin_depth=float(input("Set the basin depth (m):"))
    basin_area = basin_volume /basin_depth
    HRT_test = - (basin_volume / (0.0005 * basin_area)) * np.log((DO_s - DO_out) / (DO_s - DO))
    DOout_test=DO_s-(DO_s-DO)*mh.exp(-0.0005*basin_area/hrt*basin_volume)
    print("The testing DO is",DOout_test)
    if DOout_test>DO_out:
        print("The basin area (m2) per lane is",mh.ceil(basin_area))
    else:
        print("The  required retention time (h) should be increased")
        HRTnew=1.1*hrt
        basin_volume = flowlane * HRTnew
        basin_depth=float(input("Set the basin depth (m):"))
        basin_area = basin_volume /basin_depth
        print("The basin area (m2) is",mh.ceil(basin_area))

    if lanes>1:
            total_area=mh.ceil(basin_area*lanes)
            print("The total area of the aeration basins (m2) is:",total_area)
    else:
            total_area=basin_area
    #Calculate the pumps' characteristics
    print("Assuming 2 (1+1 stand-by) pumps per lane")
    Qpump=flowlane
    Hpump=1.1*(5) #assuming 2m of losses in frictions
    Ppump=1.1*(Qpump*Hpump/(367*0.75)) #Kw
    Epump=Ppump*lanes*24
    pumparea=4*(lanes*2) # assuming 4m2 per lane based on avarage dimensions of pumps

    # Total aerea required
    aerarea=mh.ceil(blowarea+pumparea+total_area)
    print("The total area of the aeration process (m2) is:",aerarea)

    # Total required energy
    aerenergy=mh.ceil(Eblo+Epump)
    print("The total energy consumption (kWh) per day is:",aerenergy)
    return aerarea,aerenergy